In [1]:
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd
from scipy.io import loadmat

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis 
from sklearn.neural_network import MLPClassifier

from sklearn.preprocessing import StandardScaler
from pathlib import Path, PurePath

p = Path.cwd()

In [2]:
train_ratio = 0.9
L = 3000;
ind_start = 1000
n_train = round(L*train_ratio)
n_test = round(L*(1-train_ratio) )
trainInd = np.arange(0,n_train, dtype=np.uint16)
testInd = np.arange(n_train,L, dtype=np.uint16)

In [3]:
names = ["LDA",
         "Linear SVM",
         "sigmoid kernel SVM", 
         "quadratic kernel  SVM", 
         "rbf kernel SVM", 
         "Decision Tree",
         "Neural Net logistic",
         "Neural Net LRU"]
classifiers = [LinearDiscriminantAnalysis(),
              SVC(kernel="linear"), 
              SVC(kernel="sigmoid"), 
              SVC(kernel="poly"), 
              SVC(kernel="rbf"), 
              DecisionTreeClassifier(max_depth=5),
              MLPClassifier( activation='tanh'),
              MLPClassifier( activation='relu')]

n_iter = 1
df = pd.DataFrame(index = np.arange(1,n_iter+1),columns = names)

scaler = StandardScaler()  

for name, clf in zip(names, classifiers):
    for j in np.arange(1,n_iter+1):
        # load data
        filename = 'strainSet_th0.1ph0.312it' + str(j)+ 'harm0.2'
        strainData = loadmat( PurePath(p.parent / 'eulerLagrangeData' / filename ).as_posix()  ) 

        # split data into training and testing 
        Xtrain = np.concatenate((  strainData['strain_0'][:,trainInd],  strainData['strain_10'][:,trainInd]   ) , axis=1).transpose()
        Ytrain = np.concatenate(( np.zeros(n_train), np.ones(n_train)), axis = 0)
        Xtest = np.concatenate((  strainData['strain_0'][:,testInd],  strainData['strain_10'][:,testInd]   ) , axis=1).transpose()
        Ytest = np.concatenate(( np.zeros(n_test), np.ones(n_test)), axis = 0)
        
        scaler.fit(Xtrain)  
        Xtrain = scaler.transform(Xtrain)  
        Xtest = scaler.transform(Xtest)  

        clf.fit(Xtrain,Ytrain)
        score = clf.score(Xtest,Ytest)
        df.loc[j][name] = score
        
print( ) 
print('Average accuracy over ', n_iter, 'trials') 
print( df.loc[:].mean() ,'\n' )
print( df.loc[:].std()   )

D:\D_program_files\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")



Average accuracy over  1 trials
LDA                      0.498333
Linear SVM               0.610000
sigmoid kernel SVM       0.501667
quadratic kernel  SVM    0.558333
rbf kernel SVM           0.570000
Decision Tree            0.655000
Neural Net logistic      0.901667
Neural Net LRU           0.536667
dtype: float64 

LDA                     NaN
Linear SVM              NaN
sigmoid kernel SVM      NaN
quadratic kernel  SVM   NaN
rbf kernel SVM          NaN
Decision Tree           NaN
Neural Net logistic     NaN
Neural Net LRU          NaN
dtype: float64
